In [103]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana004'

# データ抽出
> ・全体データ（all_df.pkl） <br>
> ・試合データ（game_info.pkl）

In [104]:
# 全体データ
all_df = pd.read_pickle(data_path / "all_df.pkl")

# 試合データ
game_info = pd.read_pickle(data_path / "game_info.pkl")

In [105]:
game_info

,startTime,bottomTeam,bgBottom,topTeam,place,startDayTime,bgTop,gameID
0,18:00,DeNA,3,広島,横浜,2020-06-19 18:00:00,6,20202173
1,18:00,ヤクルト,2,中日,神宮,2020-06-19 18:00:00,4,20202174
2,18:00,巨人,1,阪神,東京ドーム,2020-06-19 18:00:00,5,20202175
3,18:00,ソフトバンク,12,ロッテ,PayPayドーム,2020-06-19 18:00:00,9,20202170
4,18:00,オリックス,11,楽天,京セラD大阪,2020-06-19 18:00:00,10,20202171
...,...,...,...,...,...,...,...,...
721,13:00,ソフトバンク,12,ロッテ,PayPayドーム,2020-11-15 13:00:00,9,20203323
722,18:10,巨人,1,ソフトバンク,京セラD大阪,2020-11-21 18:10:00,12,20203326
723,18:10,巨人,1,ソフトバンク,京セラD大阪,2020-11-22 18:10:00,12,20203327
724,18:30,ソフトバンク,12,巨人,PayPayドーム,2020-11-24 18:30:00,1,20203328


In [106]:
# インデックスは重複（テストデータ結合のため）しているので、初期化
all_df = all_df.reset_index()
all_df

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN
54204,33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN
54205,33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN
54206,33806,3,1,1,False,True,False,NaN,NaN,NaN,周東 佑京,L,NaN,NaN,NaN,0,20202806,8回裏,NaN,NaN,田村 伊知郎,R,NaN,5,NaN


# 使用列
> 試合情報（game_info）<br>
> ・gameID <br>
> ・先行チーム名（topTeam）<br>
> ・後行チーム名（bottomTeam）<br>
> <br>
> 全体データ（all_df）<br>
> ・バッター名（batter）<br>
> ・ピッチャー名（pitcher）<br>
> ・試合ID（gameID）

In [107]:
# 使用列の定義
cols_game = ['gameID', 'topTeam', 'bottomTeam']

In [108]:
# 必要データの抽出
game_df = game_info[cols_game]
game_df

,gameID,topTeam,bottomTeam
0,20202173,広島,DeNA
1,20202174,中日,ヤクルト
2,20202175,阪神,巨人
3,20202170,ロッテ,ソフトバンク
4,20202171,楽天,オリックス
...,...,...,...
721,20203323,ロッテ,ソフトバンク
722,20203326,ソフトバンク,巨人
723,20203327,ソフトバンク,巨人
724,20203328,巨人,ソフトバンク


In [109]:
# 試合IDの重複確認
game_df['gameID'].nunique()

726

☝：試合IDに重複なし

In [110]:
# 使用列の定義
cols_all = ['batter', 'pitcher', 'gameID', 'inning']

In [111]:
# 必要データの抽出
_all_df = all_df[cols_all].reset_index()
_all_df

,index,batter,pitcher,gameID,inning
0,0,ピレラ,今永 昇太,20202173,1回表
1,1,ピレラ,今永 昇太,20202173,1回表
2,2,ピレラ,今永 昇太,20202173,1回表
3,3,ピレラ,今永 昇太,20202173,1回表
4,4,ピレラ,今永 昇太,20202173,1回表
...,...,...,...,...,...
54203,54203,坂口 智隆,床田 寛樹,20202023,5回表
54204,54204,メヒア,堀岡 隼人,20202640,9回表
54205,54205,鈴木 誠也,ディプラン,20202864,7回裏
54206,54206,周東 佑京,田村 伊知郎,20202806,8回裏


# データ結合
> 全体データ（all_df）に試合情報（game_df）を、試合IDで外部結合する

In [112]:
all_game_df = _all_df.merge(

                    # 結合データ
                    game_df,

                    # 結合キー（試合ID）
                    on='gameID',

                    # 結合方法（左外部結合）
                    how='left'
            )

In [113]:
all_game_df

,index,batter,pitcher,gameID,inning,topTeam,bottomTeam
0,0,ピレラ,今永 昇太,20202173,1回表,広島,DeNA
1,1,ピレラ,今永 昇太,20202173,1回表,広島,DeNA
2,2,ピレラ,今永 昇太,20202173,1回表,広島,DeNA
3,3,ピレラ,今永 昇太,20202173,1回表,広島,DeNA
4,4,ピレラ,今永 昇太,20202173,1回表,広島,DeNA
...,...,...,...,...,...,...,...
54203,54203,坂口 智隆,床田 寛樹,20202023,5回表,ヤクルト,広島
54204,54204,メヒア,堀岡 隼人,20202640,9回表,広島,巨人
54205,54205,鈴木 誠也,ディプラン,20202864,7回裏,巨人,広島
54206,54206,周東 佑京,田村 伊知郎,20202806,8回裏,西武,ソフトバンク


In [114]:
# 結合確認
all_game_df[cols_game].isnull().sum()

gameID        0
topTeam       0
bottomTeam    0
dtype: int64

☝：結合OK

# データ加工
> ・バッターのチーム名 <br>
> ・ピッチャーのチーム名 <br>

In [115]:
# イニング取得（表or裏）
import re

def get_inning(col):
    
    pattern = '\d*回(.)'
    
    result = re.match(pattern, col)
    
    if result:
        
        return result.group(1)

In [116]:
all_game_df['kbn_inning'] = all_game_df['inning'].apply(lambda x:get_inning(x))
all_game_df

,index,batter,pitcher,gameID,inning,topTeam,bottomTeam,kbn_inning
0,0,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表
1,1,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表
2,2,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表
3,3,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表
4,4,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表
...,...,...,...,...,...,...,...,...
54203,54203,坂口 智隆,床田 寛樹,20202023,5回表,ヤクルト,広島,表
54204,54204,メヒア,堀岡 隼人,20202640,9回表,広島,巨人,表
54205,54205,鈴木 誠也,ディプラン,20202864,7回裏,巨人,広島,裏
54206,54206,周東 佑京,田村 伊知郎,20202806,8回裏,西武,ソフトバンク,裏


In [117]:
all_game_df['kbn_inning'].unique()

array(['表', '裏'], dtype=object)

In [118]:
all_game_df['kbn_inning'] == '表'

0         True
1         True
2         True
3         True
4         True
         ...  
54203     True
54204     True
54205    False
54206    False
54207    False
Name: kbn_inning, Length: 54208, dtype: bool

☝：イニングの処理完了

In [119]:
# バッター（ピッチャー）チーム名
all_game_df['batterTeam'] = all_game_df.apply(
    
                                    lambda x:
    
                                            # 表の場合、バッターは先行チーム（ピッチャーは後行チーム）
                                            x['topTeam'] if x['kbn_inning'] == '表'
    
                                            # 裏の場合、バッターは後行チーム（ピッチャーは先行チーム）
                                            else x['bottomTeam']
                                    
                                    ,axis=1

                            )
all_game_df['pitcherTeam'] = all_game_df.apply(
    
                                    lambda x:
    
                                            # 表の場合、バッターは先行チーム（ピッチャーは後行チーム）
                                            x['bottomTeam'] if x['kbn_inning'] == '表'
    
                                            # 裏の場合、バッターは後行チーム（ピッチャーは先行チーム）
                                            else x['topTeam']
                                    
                                    ,axis=1

                            )

In [120]:
all_game_df

,index,batter,pitcher,gameID,inning,topTeam,bottomTeam,kbn_inning,batterTeam,pitcherTeam
0,0,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表,広島,DeNA
1,1,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表,広島,DeNA
2,2,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表,広島,DeNA
3,3,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表,広島,DeNA
4,4,ピレラ,今永 昇太,20202173,1回表,広島,DeNA,表,広島,DeNA
...,...,...,...,...,...,...,...,...,...,...
54203,54203,坂口 智隆,床田 寛樹,20202023,5回表,ヤクルト,広島,表,ヤクルト,広島
54204,54204,メヒア,堀岡 隼人,20202640,9回表,広島,巨人,表,広島,巨人
54205,54205,鈴木 誠也,ディプラン,20202864,7回裏,巨人,広島,裏,広島,巨人
54206,54206,周東 佑京,田村 伊知郎,20202806,8回裏,西武,ソフトバンク,裏,ソフトバンク,西武


# 全体データに結合
> ・結合キー：インデックス

In [121]:
all_gms_df = all_df.merge(

                # 結合データ（all_game_df）
                all_game_df[
                    # 必要な列のみ
                    ['topTeam', 'bottomTeam', 'batterTeam', 'pitcherTeam']
                ],
    
                # 結合方法（内部）
                how='inner',
    
                # キー（index）
                left_index=True,
                right_index=True
)

In [122]:
all_gms_df

,id,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y,topTeam,bottomTeam,batterTeam,pitcherTeam
0,0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0,広島,DeNA,広島,DeNA
1,1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0,広島,DeNA,広島,DeNA
2,2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0,広島,DeNA,広島,DeNA
3,3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0,広島,DeNA,広島,DeNA
4,4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0,広島,DeNA,広島,DeNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN,ヤクルト,広島,ヤクルト,広島
54204,33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN,広島,巨人,広島,巨人
54205,33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN,巨人,広島,広島,巨人
54206,33806,3,1,1,False,True,False,NaN,NaN,NaN,周東 佑京,L,NaN,NaN,NaN,0,20202806,8回裏,NaN,NaN,田村 伊知郎,R,NaN,5,NaN,西武,ソフトバンク,ソフトバンク,西武


# データ保存

In [123]:
all_gms_df.to_pickle(data_path / ('%s_all_df.pkl' % prefix))